# Statistical Hypothesis Testing
- 모집단의 모수에 관한 가설을 세우고 표본의 정보를 이용해 그 가설을 검증하는 기법

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf

%precision 3
%matplotlib inline

In [ ]:
# To check the weight of fries over 130g 
df = pd.read_csv('../data/ch11_potato.csv')
sample = np.array(df['무게']) #Weight
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [5]:
s_mean = np.mean(sample)
s_mean

128.451

## 11.1 Statistical Hypothesis Testing

### 1.1.1 flow of hypothesis testing

In [ ]:
# 1)  
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95) # 1-cdf ( 오른 쪽 꼬리 5% 지점, 상위 5% 지점)
# 표본 평균이 128.681 이하의 무게가 되는 것이 5퍼센트 이하의 확률로 발생한다.

# 2) 모평균이 130인게 맞아? 더 적은 숫자 아닐까?

128.681

- 귀무가설 (null hypothesis) : 차이가 있다, 효과가 있다  H0
- 대립가설 (alternative hypothesis) : 차이가 없다, 효과가 없다 H1
######
- reject the null hypothesis : the n.h is wrong, significant  
- accpet the null hypothesis : Can't say the n.h is wrong (=/= n.h is right)
######
- rejection region
- acceptance region
#####
- 기각역에 들어가는 확률을 정하고 검정을 수행해야한다. : 유의 수준 (level of significant) 5%
- 경계선상의 값을 임곗값 critical value 128.681
- 검정에 사용되는 통계값을 검정 통계량 test statistic sample value
####
- p_value < 유의수준 : reject null hypothesis vs accept


In [ ]:
# Test statistic sample value (standalization)
z = (s_mean - 130) / np.sqrt(9/14)
z


-1.932

In [ ]:
# Critical value
rv = stats.norm()
rv.isf(0.95) # 꼬리쪽 확률이 0.95가 되게 하는 임계값 P(Z > -1.644)

# Reject n.h and sample mean is lower than 130 g

-1.645

In [12]:
# Check the p 
rv.cdf(z)

0.027

### 11.1.2. One-tailed test & two-tailed test
- 모평균은 130g 이 아니다 라는 대립가설로 가설검정을 수행할 때도 있다
- 이의 경우에 130g보다 큰 경우도 고려하는데 이를 two-tailed test 라고 한다
- 단측 검정 측의 기각역이 더 넓다  : 양측 검정 보다 귀무가설을 기각하기 쉽다

In [13]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932

In [14]:
rv = stats.norm()
rv.interval(0.95)

(-1.960, 1.960)

In [15]:
# Two side → two times 
rv.cdf(z) * 2

0.053

### 11.1.3. Two errors
- 제 1종 오류 : 귀무가설이 옳을 때, 귀무가설을 기각하는 오류 - 위험률         - 분석가 제어 가능
- 제 2종 오류 : 대립가설일 옳을 때, 귀무가설을 채택하는 오류 - 검정력 power   - 분석가 제어 불가

In [ ]:
# 1종 오류 : 모평균이 130g인데도 평균은 130g보다 작다에 도달하는 경우 
# 모집단의 확률 분포
rv = stats.norm(130, 3)

In [ ]:
c = stats.norm().isf(0.95)
n_samples = 10000                     # 2) Testing 10000 times
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2) # 1) Select 14 sample
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1

cnt / n_samples

# False positive : 0.053
# 귀무가설이 사실인데도 기각하는 경우

0.053

- 위험률(significance level) : 1종오류를 범할 확률
- α = significiance level
- 1종 오류가 발생하는 확률을 1%로 하고 싶다면 유의 수준 1% 내에서 검증하면 됨

In [ ]:
# 2종 오류 : 모평균이 130g 보다 작다 인데도, 이 결론을 얻을 수 없는 상황 (미탐 false negative)
# 대립가설이 참인데도, 귀무가설을 채택하는 경우

# “모평균은 130보다 작다” (H1)이 사실임에도 불구하고, 표본에서 얻은 결과가 H0 채택역에 들어가서 “모평균은 130이다”라고 결론 내리는 비율
rv = stats.norm(128, 3)

c = stats.norm().isf(0.95)

n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
        
cnt / n_samples

0.195

## 11.2 Hypothesis testing

### 11.2.1 정규분포의 모평균에 대한 검정 : 모분산을 알고 있는 경우
- 모평균이 어떤 값이 아닌 것을 주장하기 위한 검정

In [22]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)

    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p값은 {p:.3f}')

In [ ]:
pmean_test(sample, 130, 9)

# Accept null hypothesis, p = 0.053

귀무가설을 채택
p값은 0.053


### 11.2.1 정규분포의 모분산에 대한 검정
- 모분산이 어떤 값이 아닌 것을 주장하기 위한 검정

In [24]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p값은 {p:.3f}')

In [25]:
pvar_test(sample, 9)

귀무가설을 채택
p값은 0.085


### 11.2.3. 정규분포의 모평균에 대한 검정(모분산을 알지 못함)
- 1-sample t-test

In [26]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p값은 {p:.3f}')

In [27]:
pmean_test(sample, 130)

귀무가설을 채택
p값은 0.169


In [28]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.455, 0.169)

## 11.3 Two-sample problem

### 1.3.1. Paired t-test 
- 대응하는 데이터가 있고, 데이터 차이에 정규분포를 가정할 수 있는 경우의 평균값 차이에 대한 검정

In [ ]:
# e.g 1 Concentration before training and after
training_rel = pd.read_csv('../data/ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head() 

(20, 2)


,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


In [ ]:
# null = uafter - ubefore = 0
# alternative = uafter - ubefore =/= 0

# 대응되는 데이터가 있는 대응 비교이므로, 각각의 데이터에서 차이를 생각할 수 있음

training_rel['차'] = training_rel['후'] - training_rel['전']
training_rel.head()

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


- 1표본 t검정임 사실상.
- 차이값에 대한 평균이 = 0 인지 검정 → 본질적으로 1표본 t-test

In [ ]:
t, p = stats.ttest_1samp(training_rel['차'], 0)
p

# reject n.h

0.040

In [33]:
t, p = stats.ttest_rel(training_rel['후'], training_rel['전'])
p

0.040

### 11.3.2. Independent t-test
- 대응하는 데이터가 없고, 독립된 2표본 모집단에 정규분포를 가정할 수 있는 경우 평균값의 차이에 대한 검정

In [67]:
# e.g Concentraion in A and B group

training_ind = pd.read_csv('../data/ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [ ]:
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'],
                       equal_var=False) # 웰치의 방법
p

0.087

### 11.3.3. Wilcoxon signed-rank test
- 대응 표본에서 차이에 정규분포를 가정할 수 없는 경우, *중앙값의 차이* 에 대한 검정

In [36]:
training_rel = pd.read_csv('../data/ch11_training_rel.csv')
toy_df = training_rel[:6].copy()
toy_df

,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


In [37]:
diff = toy_df['후'] - toy_df['전']
toy_df['차'] = diff
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [ ]:
# 순위에 의해 검정을 수행함
# 1) 차이의 절댓값(absolute value)이 작은 것부터 순위를 부여
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [ ]:
# 2) 차이의 부호가 마이너스인 것의 순위합과, 차이의 부호가 플러스 인것의 순위합을 각각 구함
# Compute the sum of ranks for differences with negative signs, and the sum of ranks for differences with positive signs separately.
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

# 3) 작은 쪽이 검정 통계량이 됨. 8!

(8, 13)

- 이 검정 통계량이 임곗값보다 작은 경우에 귀무 가설이 기각되는 단측검정을 수행한다

In [40]:
toy_df['후'] = toy_df['전'] + np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [41]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(0, 21)

In [42]:
toy_df['후'] = toy_df['전'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [43]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(11, 10)

In [44]:
T, p = stats.wilcoxon(training_rel['전'], training_rel['후'])
p

0.040

In [45]:
T, p = stats.wilcoxon(training_rel['후'] - training_rel['전'])
p

0.040

In [ ]:
# 모집단이 정규분포를 따르는 경우도 사용할 수 있으나, 대응비교 t 검정에 비해 검정력이 낮음
n = 10000
diffs = np.round(stats.norm(3, 4).rvs(size=(n, 20)))

In [47]:
cnt = 0
alpha = 0.05
for diff in diffs:
    t, p = stats.ttest_1samp(diff, 0)
    if p < alpha:
        cnt += 1
cnt / n

0.888

In [48]:
cnt = 0
alpha = 0.05
for diff in diffs:
    T, p = stats.wilcoxon(diff)
    if p < alpha:
        cnt += 1
cnt / n

0.878

### 11.3.4 Mann-Whitney rank-test

- 대응되는 데이터가 없는 2표본 모집단에 정규분포를 가정할 수 없는 경우, 중앙값 차이에 대한 검정

In [49]:
training_ind = pd.read_csv('../data/ch11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [ ]:
# 데이터 전체에 대해서 값이 작은 순서대로 순위를 부여함 
# A에 좋은 순위가 모여 잇으면 순위 합이 작아지고, 반대로 나쁜 순위가 모여 있다면 순위합이 커지는 것처럼 
# 순위합은 2표본 사이의 데이터 편향을 잘 반영한다.
rank = stats.rankdata(np.concatenate([toy_df['A'],
                                      toy_df['B']]))
rank_df = pd.DataFrame({'A': rank[:5],
                        'B': rank[5:10]}).astype(int)
rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


In [ ]:
n1 = len(rank_df['A'])

# This is the correct U 
u = rank_df['A'].sum() - (n1*(n1+1))/2 # 검정 통계량의 최솟값을 0으로 하기 위한 수 
u

7.000

In [52]:
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5).T,
                       columns=['A', 'B'])
rank_df

,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [53]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

0.000

In [54]:
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5)[::-1].T,
                       columns=['A', 'B'])
rank_df

,A,B
0,6,1
1,7,2
2,8,3
3,9,4
4,10,5


In [55]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

25.000

In [56]:
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'],
                          alternative='two-sided')
p

0.059

- 2표본의 중앙값에 편향이 있다는 사실에는 변함이 없음. 양측검정을 수행하게 됨.

### 11.3.5 Test for independence = chi-square test

In [ ]:
# eg. 어떤 상품 광고 전략으로 광고 A와 B가 만들어 졌고, 어느 광고가 더 구매 욕구를 촉진하는지 논의되고 있다.
# 이를 위해 광고 A와 광고 B를 모두 내보냈을 때 구입 비율에 유의한 차이가 있는지 확인하려면?

# 독립 : A를 내보내든, B를 내보내든 변화가 없음
# 독립 x : 둘다 내보냈을 때 상품 구입 비율에 유의한 차이가 나옴 
ad_df = pd.read_csv('../data/ch11_ad.csv')
n = len(ad_df)
print(n)
ad_df.head()

1000


,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다


In [ ]:
# Create cross table
ad_cross = pd.crosstab(ad_df['광고'], ad_df['구입'])
ad_cross

구입,하지 않았다,했다
광고,,
A,351,49
B,549,51


In [ ]:


ad_cross['했다'] / (ad_cross['했다'] + ad_cross['하지 않았다'])

광고
A    0.1225
B    0.0850
dtype: float64

In [ ]:
# 상품을 구입한 사람의 합계, 상품을 구입하지 않은 사람의 합계

n_not, n_yes = ad_cross.sum()
n_not, n_yes

(900, 100)

In [ ]:
# 광고 A를 본 사람의 합계, B를 본사람의 합계
n_adA, n_adB = ad_cross.sum(axis=1)
n_adA, n_adB

(400, 600)

In [ ]:
# 광고와 구입이 독립인 변수일 때, 기대되는 도수를 기대도수 expected frequency / 관측되는 데이터를 관측 도수 observed frequency
ad_ef = pd.DataFrame({'했다': [n_adA * n_yes / n,
                              n_adB * n_yes / n],
                      '하지 않았다': [n_adA * n_not / n,
                                   n_adB * n_not / n]},
                      index=['A', 'B'])
ad_ef

,했다,하지 않았다
A,40.0,360.0
B,60.0,540.0


In [63]:
y = ((ad_cross - ad_ef) ** 2 / ad_ef).sum().sum()
y

3.750

In [64]:
rv = stats.chi2(1)
1 - rv.cdf(y)

0.053

In [65]:
chi2, p, dof, ef = stats.chi2_contingency(ad_cross,
                                          correction=False)
chi2, p, dof

(3.750, 0.053, 1)

In [66]:
ef

array([[360.,  40.],
       [540.,  60.]])